### Dooh AR Senior D - AI Team - Barnatt Wu and Johnathan Vu
#### The following notebook is used to experiment with the yolov7-tiny model to make it compatible with our data and Snap's Lens Studio

Download data
Some links:
https://docs.ultralytics.com/guides/hyperparameter-tuning/


In [ ]:
import urllib.request, os

# download the model's weights 
# NOTE: the model is not compatible with the ultralytics library

os.makedirs("./models/yolov7_tiny/weights", exist_ok=True)
url = "https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt"
save_path = "./models/yolov7_tiny/weights/yolov7-tiny.pt"

urllib.request.urlretrieve(url, save_path)
print("Downloaded:", save_path)

In [ ]:
from ultralytics import YOLO

# import and intialize yolo model weights (.pt)
# model = YOLO("yolov8n.pt")
model = YOLO("./models/yolov8/yolov8n.pt")


# fine-tune model on training data
# TO-DO: Download and set up datasets, write yaml file, and train the model

# test, evaluate, etc.

In [ ]:
# quantize and export the model. If model size is still too big, try pruning the model
model.export(
    format="onnx",
    half=True,
    simplify=True,
    dynamic=False,
    project="./models/yolov8",
    name="yolov8n_fp16"
)

In [ ]:
# exmaple usage of the model
from PIL import Image
import matplotlib.pyplot as plt

# Load the pretrained model

# Run inference on an image
results = model("test_wtc.jpg", imgsz=640)

# Show results inline
for r in results:
    im_array = r.plot()  # render detections
    im = Image.fromarray(im_array)
    plt.imshow(im)
    plt.axis("off")
    plt.show()

url = "https://ultralytics.com/images/zidane.jpg"
results = model(url, imgsz=640)

for r in results:
    im_array = r.plot()
    im = Image.fromarray(im_array)
    plt.imshow(im)
    plt.axis("off")
    plt.show()

In [ ]:
from dotenv import load_dotenv
from roboflow import Roboflow
import os

load_dotenv()

roboflow_api_key = os.getenv("ROBOFLOW_API_KEY")

rf = Roboflow(api_key=roboflow_api_key)
project = rf.workspace().project("dooh-ar-senior-d-vnzov")
dataset = project.version(1).download("yolov8")

In [ ]:
# Fine-tune YOLOv8 model on your WTC dataset
from ultralytics import YOLO
import os

project_root = os.path.dirname(os.path.abspath(__file__)) if "__file__" in globals() else os.getcwd()
print("Current working directory:", project_root)

data_yaml_path = os.path.join(project_root, "ml_training", "data.yaml")

# Load pre-trained model
model = YOLO("./models/yolov8/yolov8n.pt")

# Train the model on your dataset
results = model.train(
    data=data_yaml_path,
    epochs=25,
    batch=16,
    lr0=0.01,
    exist_ok=True,
    plots=True,
    val=True,
)

print("Training completed!")
print(f"Results saved to: {results.save_dir}")


In [ ]:
print(results)

In [ ]:
# Export the fine-tuned model to ONNX format for deployment
model.export(
    format="onnx",
    half=True,           # Use FP16 precision for smaller model size
    simplify=True,       # Simplify the ONNX model
    dynamic=False,       # Static input shapes for better optimization
    project="./models/yolov8",
    name="yolov8n_wtc_finetuned"
)

print("Model exported to ONNX format!")


In [ ]:
# Test the fine-tuned model on your test image
from PIL import Image
import matplotlib.pyplot as plt

# Load the fine-tuned model
finetuned_model = YOLO("./models/yolov8/yolov8n_wtc_finetuned.onnx")

# Run inference on test image
results = finetuned_model("test_wtc.JPG", imgsz=640)

# Display results
for r in results:
    im_array = r.plot()  # render detections
    im = Image.fromarray(im_array)
    plt.figure(figsize=(12, 8))
    plt.imshow(im)
    plt.axis("off")
    plt.title("Fine-tuned YOLOv8 Results on WTC Dataset")
    plt.show()
    
    # Print detection results
    print("Detection Results:")
    for box in r.boxes:
        class_id = int(box.cls[0])
        confidence = float(box.conf[0])
        print(f"Class: {r.names[class_id]}, Confidence: {confidence:.3f}")
